While in example #1 we compare two models based on our data, in this step we will choose the better model, create predictions, and insert these predictions into SQL Server

Note: while we created and checked models in step #1, we automatically held out some of the data from your query in a test set (to see how accurate our models were). Know that we have a sense for model accuracy, in this step, we re-create the best model on your ENTIRE data set, and then use it to create predictions.

If you have SQL Server on your machine and HCRTools installed, there's just one more prerequisite: an entity to receive the predictions. Call it what you may, but it should have these cols:

In [94]:
#CREATE TABLE [SAM].[dbo].[HCRDeployBASE](      --If possible, create using SAMD
#[BindingID] [int] NOT NULL,
#[BindingNM] [varchar](255) NOT NULL,
#[LastLoadDTS] [datetime2](7) NOT NULL,
#[GrainID] [decimal](38, 0) NOT NULL,          --Make this col name same as in input table
#[PredictedProbNBR] [decimal](38, 2) NOT NULL, --Change to PredictedValueNBR for regression
#[Factor1TXT] [varchar](255) NULL,
#[Factor2TXT] [varchar](255) NULL,
#[Factor3TXT] [varchar](255) NULL
#)

Now on to the code! First, load the package and create a connection to SQL Server.

In [95]:
library(HCRTools)

connection.string <- 'driver={SQL Server};
                      server=localhost;
                      database=SAM;
                      trusted_connection=true'

Let's bring in the data.

In [96]:
query <- "SELECT 
      [GrainID]
      ,[MaritalStatus]
      ,[Gender]
      ,[SalariedFlag]
      ,[SickLeaveHours]
      ,[InTestWindow]   --Note this col should be Y/N
  FROM [SAM].[dbo].[HREmployeeDeploy]"
df <- SelectData(connection.string, query)

Check the data types of the dataframe to make sure factor cols aren’t listed as numeric cols, etc.

In [97]:
str(df)

'data.frame':	290 obs. of  6 variables:
 $ GrainID       : int  10001 10002 10003 10004 10005 10006 10007 10008 10009 10010 ...
 $ MaritalStatus : Factor w/ 2 levels "M","S": 2 2 NA 2 1 1 1 2 1 1 ...
 $ Gender        : Factor w/ 2 levels "F","M": 2 1 2 2 1 2 2 1 1 2 ...
 $ SalariedFlag  : Factor w/ 2 levels "N","Y": 2 2 NA 1 2 2 2 2 NA 2 ...
 $ SickLeaveHours: int  69 NA 21 NA 22 23 50 51 51 64 ...
 $ InTestWindow  : Factor w/ 2 levels "N","Y": 2 2 2 2 2 2 2 2 2 2 ...


Change a column type, if necessary

In [98]:
df$SalariedFlag = as.factor(df$SalariedFlag)    # only here for demonstration
df$SickLeaveHours = as.numeric(df$SickLeaveHours) # only here for demonstration
str(df)

'data.frame':	290 obs. of  6 variables:
 $ GrainID       : int  10001 10002 10003 10004 10005 10006 10007 10008 10009 10010 ...
 $ MaritalStatus : Factor w/ 2 levels "M","S": 2 2 NA 2 1 1 1 2 1 1 ...
 $ Gender        : Factor w/ 2 levels "F","M": 2 1 2 2 1 2 2 1 1 2 ...
 $ SalariedFlag  : Factor w/ 2 levels "N","Y": 2 2 NA 1 2 2 2 2 NA 2 ...
 $ SickLeaveHours: num  69 NA 21 NA 22 23 50 51 51 64 ...
 $ InTestWindow  : Factor w/ 2 levels "N","Y": 2 2 2 2 2 2 2 2 2 2 ...


Do pre-processing and create object with data split into train and test sets.

In [99]:
o <- DeploySupervisedModel$new(type = 'classification',
                               df = df,
                               grain.col = 'GrainID',
                               window.col = 'InTestWindow',
                               predicted.col = 'Gender',
                               impute = TRUE,
                               debug = FALSE,
                               use.saved.model = FALSE)

Note that in Example #1 we found the logistic model to work best, so let's use it to create predictions. This step will do that and push the results to SQL Server (note you choose the destination entity here).

In [100]:
o$deploy(model = 'logit',  # Change to 'rf' for random forest
         cores = 1,
         sqlcnxn = connection.string,
         debug = FALSE,    # Change this to TRUE to debug
         # Note: Do not use [ or ] in output table
         dest.schema.table = 'dbo.HCRPredictionBASE')

[1] "Details for proability model:"

Call:  glm(formula = as.formula(paste(self$predicted.col, ".", sep = " ~ ")), 
    family = binomial(link = "logit"), data = self$dfTrain, control = list(maxit = 10000), 
    metric = "ROC", trControl = trainControl(classProbs = TRUE, 
        summaryFunction = twoClassSummary))

Coefficients:
    (Intercept)  MaritalStatus.S   SalariedFlag.Y   SickLeaveHours  
        1.64261          0.34380         -0.93506         -0.01441  

Degrees of Freedom: 252 Total (i.e. Null);  249 Residual
Null Deviance:	    296.5 
Residual Deviance: 287.7 	AIC: 295.7
[1] "Probability predictions are based on logistic"
[1] "SQL Server insert was successful"


That's it! Your choice of model has been trained on your entire dataset, your predictions have been made, and your results now sit in SQL Server. Remember that to see the docs you can always type this in the console: 

In [101]:
library(HCRTools)
?HCRTools

Feel reach out to Levi Thatcher (levi.thatcher@healthcatalyst.com) with any questions or suggestions!